In [ ]:
# Import packages 
import os
import sys

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.tri as tri
import matplotlib.colors as colors

from scipy.io import FortranFile
from scipy.interpolate import LinearNDInterpolator as Li
from scipy.interpolate import NearestNDInterpolator as Ni
from scipy.spatial import Delaunay as Dl

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import importlib
import glob
import copy

In [ ]:
# ----------------------------------
# Functions to calculate area of
# triangle on a sphere
#-----------------------------------
def latlong_to_cartesian(latlong):
    phi   = np.deg2rad( 90 - latlong[0] )
    theta = np.deg2rad( latlong[1] )
    
    x = np.sin(phi) * np.cos(theta)
    y = np.sin(phi) * np.sin(theta)
    z = np.cos(phi)
    return np.array([x,y,z])
    
def spherical_area(a, b, c):
    t = np.abs( np.inner(a, np.cross(b, c) ) )
    t /= 1 + np.inner(a,b) + np.inner(b,c) + np.inner(a,c)
    return 2*np.arctan(t)
    
def flat_area(a, b, c):
    x = np.cross(a - b, c - b)
    return 0.5*np.linalg.norm(x)


In [ ]:
#scripF='/glade/p/cesmdata/inputdata/share/scripgrids/C540_SCRIP_desc.230330.nc'
scripF='PE540x3240-CF.nc4'
dS=xr.open_dataset(scripF)

In [ ]:
print(list(dS.variables))

In [ ]:
lon=dS['grid_center_lon'].values
lat=dS['grid_center_lat'].values
crlon=dS['grid_corner_lon'].values
crlat=dS['grid_corner_lat'].values
area_on_file=dS['grid_area'].values


lonx0=crlon[:,0]
latx0=crlat[:,0]
lonx1=crlon[:,1]
latx1=crlat[:,1] 
lonx2=crlon[:,2]
latx2=crlat[:,2]
lonx3=crlon[:,3]
latx3=crlat[:,3] 

In [ ]:
print(np.sum(area_on_file) , 4.*np.pi )

In [ ]:
nop=len(lon)

grid_area=np.zeros( nop )
for i in np.arange(nop ):
    A  = ( latx0[i],  lonx0[i] )
    B  = ( latx1[i],  lonx1[i] )
    C  = ( latx2[i],  lonx2[i] )
    D  = ( latx3[i],  lonx3[i] )

    a = latlong_to_cartesian(A)
    b = latlong_to_cartesian(B)
    c = latlong_to_cartesian(C)
    d = latlong_to_cartesian(D)
    
    round1 = spherical_area(a, b, c)
    round2 = spherical_area(c, d, a)
    grid_area[i] = round1 + round2
    if ((i % 100_000 )==0):
        print(i,nop)


In [ ]:
print(np.sum(grid_area) , 4.*np.pi )

## A good place to STOP for a while

In [ ]:
refinement_c = np.sqrt( np.max(grid_area) / grid_area )
irefinement_c=np.round(refinement_c)

In [ ]:
print(np.min(irefinement_c))

In [ ]:
dCorr = copy.deepcopy( dS )
Dar1 = xr.DataArray( data=grid_area , dims=['grid_size']  , 
                    attrs=dict( description='Grid_cell_area',units='Steradians',) ,) 

dCorr['grid_area']=Dar1

Dar2 = xr.DataArray( data=irefinement_c , dims=['grid_size']  , 
                    attrs=dict( description='Integer_Refinement_Factor',units='1',) ,) 
    
dCorr['grid_refinement_factor']=Dar2

dCorr.to_netcdf( 'C540_CONUS_SCRIP_area.nc' )

In [ ]:
xlat=np.linspace(-90,90,num=181)
xlon=np.linspace(0,360,num=360)
X,Y=np.meshgrid(xlon,xlat)
print(X.shape)

ooo=np.c_[lon,lat]
dlo=Dl(ooo)

# Interpolote ocean mask to lat lon
grinx = Ni( dlo, grid_area )
grid_area_xy = grinx( X, Y )

In [ ]:
##
gronx = Ni( dlo, area_on_file )
area_on_file_xy = gronx( X, Y )

In [ ]:
clev=np.logspace(-7,-4.2,num=51)
fig = plt.figure(figsize=(15,10))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180.))
ax.coastlines(color="white")

cf = ax.contourf(xlon, xlat, grid_area_xy ,  levels=clev  , transform=ccrs.PlateCarree() , cmap="gist_ncar")
ax.set_title( "C540-CONUS grid area (steradians)")
plt.colorbar(cf,ax=ax,shrink=.5,label='Steradians')

In [ ]:
clev=np.logspace(-7,-4.2,num=51)
fig = plt.figure(figsize=(15,10))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180.))
ax.coastlines(color="white")

cf = ax.contourf(xlon, xlat, area_on_file_xy ,  levels=30  , transform=ccrs.PlateCarree() , cmap="gist_ncar")
ax.set_title( "C540-CONUS grid area (steradians)")
plt.colorbar(cf,ax=ax,shrink=.5,label='Steradians')

In [ ]:
refinement= np.sqrt( np.max(grid_area_xy) / grid_area_xy )
irefinement=np.round(refinement)
clev=np.linspace(0,10,num=21)
fig = plt.figure(figsize=(15,10))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180.))
ax.coastlines(color="black")

cf = ax.contourf(xlon, xlat, refinement ,  levels=clev  , transform=ccrs.PlateCarree() , cmap="gist_ncar")
ax.set_title( "C540 Refinement factor")
plt.colorbar(cf,ax=ax,shrink=.5,label='Refinement')

In [ ]:
Rearth = 6378.1
grid_length = np.sqrt((Rearth**2)* grid_area_xy )
clev=np.linspace(0,50,num=51)
fig = plt.figure(figsize=(15,10))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180.))
ax.coastlines(color="black")

cf = ax.contourf(xlon, xlat, grid_length ,  levels=clev  , transform=ccrs.PlateCarree() , cmap="gist_ncar")
ax.set_title( "C540-CONUS Gridlength scale ")
plt.colorbar(cf,ax=ax,shrink=.5,label='Length (km)')

In [ ]:
Rearth = 6378.1
grid_length = np.sqrt((Rearth**2)* grid_area_xy )
clev=np.linspace(-1,10,num=12)
fig = plt.figure(figsize=(15,10))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180.))
ax.coastlines(color="black")

cf = ax.contourf(xlon, xlat, irefinement ,  levels=clev  , transform=ccrs.PlateCarree() , cmap="gist_ncar")
ax.set_title( "C540-CONUS Gridlength scale ")
plt.colorbar(cf,ax=ax,shrink=.5,label='Length (km)')

In [ ]:
scripF='/glade/p/cesmdata/inputdata/share/scripgrids/C540_SCRIP_desc.230330.nc'
dS=xr.open_dataset(scripF)

print(dS['grid_center_lat'].coords)
print(dS['grid_center_lat'].dims)


In [ ]:
print(list(dCorr.variables))

In [ ]:
clev=np.linspace(1e-6,10e-6,num=31)
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180.))
ax.coastlines()
plt.tricontourf( lon,lat, grid_area , levels=clev, alpha=.5)
plt.colorbar()

In [ ]:
#plt.tricontourf( d['grid_center_lon'] , d['grid_center_lat'] , d['grid_area'] )
cc=np.max(area)/area
#cc=np.max(grid_area)/grid_area
plt.get_cmap("jet")
idx=np.arange(start=0,stop=len(lon),step=100)
#plt.plot( lon , lat,'.' ,markersize=.1,markevery=10,c=cc) # ,cmap=plt.get_cmap("jet"))

oo=np.where( (abs(lon-232.8)<.25) & (abs(lat-37.9)<.25) )
idx=oo[0][0]
plt.scatter( lon[idx] , lat[idx]  ,s=50 ,cmap="cool",c=cc[idx] )
plt.colorbar()
#plt.scatter( lon[idx] , lat[idx]  ,s=50 ,color='black') #,cmap="cool",c=cc[idx] )
plt.scatter( lonx0[idx] , latx0[idx] ,marker='v' ,s=50 ,color='red') #,cmap="cool",c=cc[idx] )
plt.scatter( lonx1[idx] , latx1[idx] ,marker='^' ,s=100 ,color='green') #,cmap="cool",c=cc[idx] )
plt.scatter( lonx2[idx] , latx2[idx] ,marker='+' ,s=50 ,color='blue') #,cmap="cool",c=cc[idx] )
plt.scatter( lonx3[idx] , latx3[idx] ,marker= 'x' ,s=50 ,color='orange') #,cmap="cool",c=cc[idx] )
plt.xlim(232.75,233.25)
plt.ylim(38,38.25)
"""
plt.scatter( lon[idx] , lat[idx]  ,s=50 ,cmap="cool",c=cc[idx] )
plt.colorbar()
"""
#plt.xlim(232.,234)
#plt.ylim(37.,39)
#plt.plot(area)

In [ ]:
#from numpy import *
    
    
# Latitude and longitude
austin  = (30.2672,  97.7431)
boston  = (42.3584,  71.0598)
calgary = (51.0501, 114.0853)
    
a = latlong_to_cartesian(austin)
b = latlong_to_cartesian(boston)
c = latlong_to_cartesian(calgary)
    
round = spherical_area(a, b, c)
flat  = flat_area(a, b, c)
print(round, flat, round/flat)